# Preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl ; mpl.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt ; plt.rcParams['font.family'] = 'HYGothic-Medium'
import seaborn as sns

import variable as var

---

In [2]:
# Train Data 불러오기 및 오류 데이터 제거
train = pd.read_csv('train.csv')
train_error = ['C2085', 'C1397', 'C2431', 'C1649', 'C1036', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
train = train[~train['단지코드'].isin(train_error)].reset_index(drop=True)
print(train.shape)

(2869, 15)


In [3]:
# Test Data 불러오기 및 오류 데이터 제거
test = pd.read_csv('test.csv')
test_error = ['C2675', 'C2335', 'C1327']
test = test[~test['단지코드'].isin(test_error)].reset_index(drop=True)
print(test.shape)

(1008, 14)


---

In [4]:
# Train Data의 '상가수' 변수 생성
store = train[train['임대건물구분'] == '상가'].groupby('단지코드', as_index=False).agg({'임대건물구분':'count'}).rename(columns={'임대건물구분':'상가수'})
train = pd.merge(train, store, how='left')
train['상가수'] = train['상가수'].fillna(0)

In [5]:
# Test Data의 '상가수' 변수 생성
store = test[test['임대건물구분'] == '상가'].groupby('단지코드', as_index=False).agg({'임대건물구분':'count'}).rename(columns={'임대건물구분':'상가수'})
test = pd.merge(test, store, how='left')
test['상가수'] = test['상가수'].fillna(0)

In [6]:
# Train Data의 '임대건물구분 == 상가' 데이터 제거, '임대건물구분' 변수 제거
train = train[train['임대건물구분'] != '상가'].reset_index(drop=True)
train.drop(['임대건물구분'], axis=1, inplace=True)
print(train.shape)

(2307, 15)


In [7]:
# Test Data의 '임대건물구분 == 상가' 데이터 제거, '임대건물구분' 변수 제거
test = test[test['임대건물구분'] != '상가'].reset_index(drop=True)
test.drop(['임대건물구분'], axis=1, inplace=True)
print(test.shape)

(831, 14)


---

In [8]:
train2 = train.copy()
train2 = var.major_voting(train2, '자격유형')
train2 = var.major_voting(train2, '공급유형')
train2 = train2[['단지코드', '자격유형_major', '공급유형_major']].drop_duplicates().reset_index(drop=True)
train = pd.merge(train, train2, on='단지코드', how='left')

In [9]:
test2 = test.copy()
test2 = var.major_voting(test2, '자격유형')
test2 = var.major_voting(test2, '공급유형')
test2 = test2[['단지코드', '자격유형_major', '공급유형_major']].drop_duplicates().reset_index(drop=True)
test = pd.merge(test, test2, on='단지코드', how='left')

---

In [10]:
# Train Data의 '전용면적별세대수합' 변수 생성
noh_by_area = train.groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합'})
train = pd.merge(left=train, right=noh_by_area, how='left', on='단지코드')
print(train.shape)

(2307, 18)


In [11]:
# Test Data의 '전용면적별세대수합' 변수 생성
noh_by_area = test.groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합'})
test = pd.merge(left=test, right=noh_by_area, how='left', on='단지코드')
print(test.shape)

(831, 17)


#### 단지명, 주소 추가하기 위해 전용면적별세대수_myhome추가

In [12]:
train_myhome = train[train['공급유형']!='공공분양']
noh_by_area = train_myhome.groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합_myhome'})
train = pd.merge(left=train, right=noh_by_area, how='left', on='단지코드')

In [13]:
test_myhome = test[test['공급유형']!='공공분양']
noh_by_area = test_myhome.groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합_myhome'})
test = pd.merge(left=test, right=noh_by_area, how='left', on='단지코드')

---

In [14]:
# print(set(train['지역'].unique()) - set(test['지역'].unique()))
# print(set(train['자격유형'].unique()) - set(test['자격유형'].unique()))
# print(set(train['공급유형'].unique()) - set(test['공급유형'].unique()))

In [15]:
# # Test Data에 없는 카테고리 Train Data에서 제거
# train = train[train['지역'] != '서울특별시'].reset_index(drop=True)
# train = train[~train['자격유형'].isin(['F', 'O', 'B'])].reset_index(drop=True)
# train = train[~train['공급유형'].isin(['공공분양', '공공임대(5년)', '장기전세'])].reset_index(drop=True)
# print(train.shape)

---

In [16]:
# Train Data의 '임대보증금', '임대료'의 '-'를 0으로 변환
train['임대보증금'] = train['임대보증금'].replace('-', 0).astype('float64')
train['임대료'] = train['임대료'].replace('-', 0).astype('float64')

In [17]:
# Test Data의 '임대보증금', '임대료'의 '-'를 0으로 변환
test['임대보증금'] = test['임대보증금'].replace('-', 0).astype('float64')
test['임대료'] = test['임대료'].replace('-', 0).astype('float64')

---

In [18]:
# Train Data의 '도보 10분거리 내 지하철역 수(환승노선 수 반영)' NA 0으로 대체
train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].fillna(0)

In [19]:
# Test Data의 '도보 10분거리 내 지하철역 수(환승노선 수 반영)' NA 0으로 대체
test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].fillna(0)

---

In [20]:
# Test Data의 '단지코드' NA 특정값으로 대체
test.loc[test['단지코드'] == 'C2411', '자격유형'] = 'A'
test.loc[test['단지코드'] == 'C2253', '자격유형'] = 'D'

---

In [21]:
# Test Data의 '임대보증금', '임대료' NA 특정값으로 대체
test.loc[test['임대보증금'].isnull(), '임대보증금'] = [5787000.0, 5787000.0, 11574000.0]
test.loc[test['임대료'].isnull(), '임대료'] = [79980.0, 79980.0, 159960.0]

---

In [22]:
train2 = train.copy()
test2 = test.copy()

In [23]:
# '단지명' 변수 생성
data = pd.read_csv('임대주택_단지_조회.csv')
data.drop(['순번', '전체건수', '최초입주년월'], axis=1, inplace=True)
data['전용면적'] = np.round(data['전용면적'], 2)
data['지역'] = ['세종특별자치시' if i[0] == '세종시' else i[0] for i in data['지역'].str.split()]

noh_by_area = data.groupby('단지명', as_index=False).agg({'전용면적별세대수':'sum'})
noh_by_area.columns = ['단지명', '전용면적별세대수합']
data = pd.merge(left=data, right=noh_by_area, how='left', on='단지명')
data.drop(['총세대수'], axis=1, inplace=True)

train2_wo_dup = train2.drop_duplicates(subset=['단지코드'])[['단지코드', '지역', '전용면적별세대수합']].reset_index(drop=True)
idx = np.column_stack([train2.drop_duplicates(subset=['단지코드'])[['지역', '전용면적별세대수합']].reset_index(drop=True), range(len(train2_wo_dup))])

train2_wo_dup['단지명'] = None
for i, j, k in idx:
    if data.loc[(data['지역'] == i) & (data['전용면적별세대수합'] == j), '단지명'].nunique() == 1:
        train2_wo_dup.iloc[k, train2_wo_dup.columns.get_loc('단지명')] = data.loc[(data['지역'] == i) & (data['전용면적별세대수합'] == j), '단지명'].unique()[0]
        
null = train2_wo_dup[train2_wo_dup['단지명'].isnull()].reset_index(drop=True)
data_hhset = data[['단지명', '전용면적']].groupby(['단지명'])['전용면적'].apply(list)
data_hhset = pd.DataFrame(data_hhset)
data_hhset = pd.merge(data_hhset, data[['단지명', '지역', '전용면적별세대수합']], on='단지명')
data_hhset = data_hhset[['단지명', '지역', '전용면적별세대수합', '전용면적']]
data_hhset = data_hhset.drop_duplicates(['단지명']).reset_index(drop=True)

idx = null['단지코드'].unique()
for i in range(len(idx)):
    areaset_train2 = set(train2.loc[train2['단지코드'] == idx[i], '전용면적'])
    region = null.loc[i, '지역']
    total_hh2 = null.loc[i,'전용면적별세대수합']
    same = data_hhset.loc[(data_hhset['지역'] == region) & (data_hhset['전용면적별세대수합'] == total_hh2)].reset_index(drop=True)
    diff = []
    if len(same.index) != 0:
        for j in range(len(same)):
            areaset_data = set(same.loc[j, '전용면적'])
            diff.append(len(areaset_train2 - areaset_data))
        null.loc[i, '단지명'] = same.loc[np.argmin(diff), '단지명']


for i in range(len(train2_wo_dup)):
    if train2_wo_dup.iloc[i, train2_wo_dup.columns.get_loc('단지명')] == None:
        train2_wo_dup.iloc[i, train2_wo_dup.columns.get_loc('단지명')] = null.loc[null['단지코드'] == train2_wo_dup.iloc[i, train2_wo_dup.columns.get_loc('단지코드')], '단지명']
    else:
        pass

train2 = pd.merge(train2, train2_wo_dup[['단지코드', '단지명']], how='left', on='단지코드')

code_name = train2['단지코드'].unique()
for i in range(len(code_name)):
    dat = train2.loc[train2['단지코드'] == code_name[i],:]
    if (dat.shape[0] != dat.단지명.isnull().sum()) & (dat.shape[0] != dat.단지명.notnull().sum()):
        train2.loc[train2['단지코드'] == code_name[i],'단지명'] = dat.loc[dat['단지명'].notnull(),'단지명']
    else:
        pass

train2.reset_index(drop=True, inplace=True)
print(round(train2.loc[train2['단지명'].isnull(), '단지코드'].nunique()/train2['단지코드'].nunique(), 2)*100)

10.0


In [24]:
# '단지명' 변수 생성
data = pd.read_csv('임대주택_단지_조회.csv')
data.drop(['순번', '전체건수', '최초입주년월'], axis=1, inplace=True)
data['전용면적'] = np.round(data['전용면적'], 2)
data['지역'] = ['세종특별자치시' if i[0] == '세종시' else i[0] for i in data['지역'].str.split()]

noh_by_area = data.groupby('단지명', as_index=False).agg({'전용면적별세대수':'sum'})
noh_by_area.columns = ['단지명', '전용면적별세대수합']
data = pd.merge(left=data, right=noh_by_area, how='left', on='단지명')
data.drop(['총세대수'], axis=1, inplace=True)

test2_wo_dup = test2.drop_duplicates(subset=['단지코드'])[['단지코드', '지역', '전용면적별세대수합']].reset_index(drop=True)
idx = np.column_stack([test2.drop_duplicates(subset=['단지코드'])[['지역', '전용면적별세대수합']].reset_index(drop=True), range(len(test2_wo_dup))])

test2_wo_dup['단지명'] = None
for i, j, k in idx:
    if data.loc[(data['지역'] == i) & (data['전용면적별세대수합'] == j), '단지명'].nunique() == 1:
        test2_wo_dup.iloc[k, test2_wo_dup.columns.get_loc('단지명')] = data.loc[(data['지역'] == i) & (data['전용면적별세대수합'] == j), '단지명'].unique()[0]
        
null = test2_wo_dup[test2_wo_dup['단지명'].isnull()].reset_index(drop=True)
data_hhset = data[['단지명', '전용면적']].groupby(['단지명'])['전용면적'].apply(list)
data_hhset = pd.DataFrame(data_hhset)
data_hhset = pd.merge(data_hhset, data[['단지명', '지역', '전용면적별세대수합']], on='단지명')
data_hhset = data_hhset[['단지명', '지역', '전용면적별세대수합', '전용면적']]
data_hhset = data_hhset.drop_duplicates(['단지명']).reset_index(drop=True)

idx = null['단지코드'].unique()
for i in range(len(idx)):
    areaset_test2 = set(test2.loc[test2['단지코드'] == idx[i], '전용면적'])
    region = null.loc[i, '지역']
    total_hh2 = null.loc[i,'전용면적별세대수합']
    same = data_hhset.loc[(data_hhset['지역'] == region) & (data_hhset['전용면적별세대수합'] == total_hh2)].reset_index(drop=True)
    diff = []
    if len(same.index) != 0:
        for j in range(len(same)):
            areaset_data = set(same.loc[j, '전용면적'])
            diff.append(len(areaset_test2 - areaset_data))
        null.loc[i, '단지명'] = same.loc[np.argmin(diff), '단지명']


for i in range(len(test2_wo_dup)):
    if test2_wo_dup.iloc[i, test2_wo_dup.columns.get_loc('단지명')] == None:
        test2_wo_dup.iloc[i, test2_wo_dup.columns.get_loc('단지명')] = null.loc[null['단지코드'] == test2_wo_dup.iloc[i, test2_wo_dup.columns.get_loc('단지코드')], '단지명']
    else:
        pass

test2 = pd.merge(test2, test2_wo_dup[['단지코드', '단지명']], how='left', on='단지코드')

code_name = test2['단지코드'].unique()
for i in range(len(code_name)):
    dat = test2.loc[test2['단지코드'] == code_name[i],:]
    if (dat.shape[0] != dat.단지명.isnull().sum()) & (dat.shape[0] != dat.단지명.notnull().sum()):
        test2.loc[test2['단지코드'] == code_name[i],'단지명'] = dat.loc[dat['단지명'].notnull(),'단지명']
    else:
        pass

test2.reset_index(drop=True, inplace=True)
print(round(test2.loc[test2['단지명'].isnull(), '단지코드'].nunique()/test2['단지코드'].nunique(), 2)*100)

7.000000000000001


In [25]:
# 입주후기간(월) 변수추가 위해 in_date 데이터 생성
data = pd.read_csv('임대주택_단지_조회.csv')
data.drop(['순번', '전체건수'], axis=1, inplace=True)
data['전용면적'] = np.round(data['전용면적'], 2)
data['지역'] = ['세종특별자치시' if i[0] == '세종시' else i[0] for i in data['지역'].str.split()]

# '단지명'과 함께 최초입주월 현재까지의 시간상 차이를 월단위로 나타내는 '입주후기간(월)'변수 생성
in_date = data[['단지명', '최초입주년월']].drop_duplicates().reset_index(drop=True)
in_date.loc[(in_date['최초입주년월']<198001)|(in_date['최초입주년월']>202501),'최초입주년월']=np.nan
in_date.loc[in_date['최초입주년월'].notnull(), '년'] = in_date.loc[in_date['최초입주년월'].notnull(), '최초입주년월'] //100
in_date.loc[in_date['최초입주년월'].notnull(), '월'] = in_date.loc[in_date['최초입주년월'].notnull(), '최초입주년월'] % 100
#입주후기간: 현재(2021.07)와 각 단지의 최초입주년월의 차이를 개월수로 표현
now = 2021*12+ 7 
in_date.loc[in_date['최초입주년월'].notnull(), '입주후기간(월)'] = now - (in_date.loc[in_date['최초입주년월'].notnull(), '년']*12 + in_date.loc[in_date['최초입주년월'].notnull(), '월'])
in_date = in_date.drop(['최초입주년월','년', '월'], axis=1)
in_date.loc[in_date['입주후기간(월)']<=0,'입주후기간(월)']=np.nan

In [26]:
# 입주후기간(월) 변수추가 
train2 = pd.merge(train2, in_date, on ='단지명', how='left')
train2 = train2.drop('단지명', axis=1)
test2 = pd.merge(test2, in_date, on = '단지명', how='left')
test2 = test2.drop('단지명', axis=1)
train2 = train2[['단지코드', '입주후기간(월)']].drop_duplicates().reset_index(drop=True)
test2 = test2[['단지코드', '입주후기간(월)']].drop_duplicates().reset_index(drop=True)

In [27]:
train = pd.merge(train, train2, on='단지코드', how='left')
test = pd.merge(test, test2, on='단지코드', how='left')

print(train.shape); print(test.shape)

(2307, 20)
(831, 19)


---

#### 단지명, 주소, 총입주민수, 준공후기간 추가

In [28]:
def total_member(df):
    age_gender = pd.read_csv('한국토지주택공사_임대주택 단지별 연령대별 성별정보_20210511.csv', encoding='CP949')
    age_gender_info = pd.read_csv('age_gender_info.csv')
    age_gender = age_gender[age_gender['주택유형']=='아파트']
    age_gender = age_gender.iloc[:,5:].groupby(['도로명주소']).sum().reset_index().merge(age_gender[['주택명','도로명주소']].drop_duplicates().groupby(['도로명주소']).first().reset_index(), on='도로명주소', how='left')
    age_gender.rename(columns={'도로명주소':'주소', '주택명':'단지명'}, inplace=True)
    age_gender = age_gender[age_gender.주소.notnull()]
    age_gender = age_gender.drop_duplicates().reset_index(drop=True) #총세대수0인거 왜날림?
    age_gender = age_gender.drop(['총세대수'], axis=1)
    age_gender['총입주민수'] = age_gender.set_index(['단지명','주소']).apply(sum,axis=1).values
    age_gender = age_gender.drop(['단지명'], axis=1)
    age_gender.loc[:,age_gender.columns.str.contains('대')] = age_gender.loc[:,age_gender.columns.str.contains('대')].mul(1/age_gender['총입주민수'], axis=0)
    df = pd.merge(df, age_gender, on=['주소'], how='left')
    na_not_df = df[df[df.columns[df.columns.str.contains('0대')]].notnull().all(axis=1)]
    na_df = df[df[df.columns[df.columns.str.contains('0대')]].isnull().any(axis=1)].loc[:,~df.columns.str.contains('0대')]
    na_df = na_df.reset_index().merge(age_gender_info, on='지역').set_index('index')
    na_df = na_df.drop(['총입주민수'], axis=1)
    na_df['총입주민수'] = na_df['총세대수'] * 1.8
    df = pd.concat([na_df, na_not_df],axis=0).sort_index().reset_index(drop=True)
    return(df)

In [29]:
# 준공공사일 후 시간 흐름 추가
def add_build_date(df):
    df.loc[df['단지명']=='가재마을 1단지', '준공일자']='2016-11-10'
    #준공일자 글자길이 7이면 '-01' 추가해주기
    df.loc[df['준공일자'].str.len()==7, '준공일자'] = df.loc[df['준공일자'].str.len()==7, '준공일자']+str('-01')
    #준공일자 글자길이 5이면 ''-01-01'추가해주기
    df.loc[df['준공일자'].str.len()==5, '준공일자'] = df.loc[df['준공일자'].str.len()==5, '준공일자']+str('01-01')
    
    df.loc[df['준공일자'].notnull(),'준공일자'] = df.loc[df['준공일자'].notnull(),'준공일자'].str.replace('-','').astype('int64')
    #준공일자 지나치게 오래된것은 오류라고 판단, NA로 대체해주기.
    df.loc[df['준공일자']<=19700101, '준공일자'] = np.nan
    
    df.loc[df['준공일자'].notnull(),'년'] = df.loc[df['준공일자'].notnull(),'준공일자']//10000
    df.loc[df['준공일자'].notnull(),'월'] = df.loc[df['준공일자'].notnull(),'준공일자']//100 - df.loc[df['준공일자'].notnull(),'년']*100
    df.loc[df['준공일자'].notnull(),'준공후기간(월)'] = 2021*12 + 7 - (df.loc[df['준공일자'].notnull(),'년']*12 + df.loc[df['준공일자'].notnull(),'월'])
    df = df.drop(['준공일자', '년', '월'], axis=1)
    return(df)

In [30]:
train2 = train.copy()
train2 = var.code_name(train2)
train2 = total_member(train2)
train2 = add_build_date(train2)
train2 = train2[['단지코드', '단지명', '주소', '총입주민수', '준공후기간(월)']].drop_duplicates().reset_index(drop=True)
train = pd.merge(train, train2, on='단지코드', how='left')

test2 = test.copy()
test2 = var.code_name(test2)
test2 = total_member(test2)
test2 = add_build_date(test2)
test2 = test2[['단지코드', '단지명', '주소', '총입주민수','준공후기간(월)' ]].drop_duplicates().reset_index(drop=True)
test = pd.merge(test, test2, on='단지코드', how='left')

print(train.shape)
print(test.shape)

(2307, 24)
(831, 23)


In [31]:
train.loc[(train['준공후기간(월)'].isnull())&(train['입주후기간(월)'].notnull()), '준공후기간(월)']= train.loc[(train['준공후기간(월)'].isnull())&(train['입주후기간(월)'].notnull()), '입주후기간(월)']
test.loc[(test['준공후기간(월)'].isnull())&(test['입주후기간(월)'].notnull()), '준공후기간(월)']= test.loc[(test['준공후기간(월)'].isnull())&(test['입주후기간(월)'].notnull()), '입주후기간(월)']

---

In [32]:
# Train Data의 '차량미보유세대수' 변수 생성
noh_wo_cars = train[train['공급유형'].isin(['영구임대', '행복주택'])].groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'차량미보유세대수'})
train = pd.merge(train, noh_wo_cars, how='left')
train['차량미보유세대수'] = train['차량미보유세대수'].fillna(0)
train.drop(['공급유형', '자격유형'], axis=1, inplace=True)

In [33]:
# Test Data의 '차량미보유세대수' 변수 생성
noh_wo_cars = test[test['공급유형'].isin(['영구임대', '행복주택'])].groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'차량미보유세대수'})
test = pd.merge(test, noh_wo_cars, how='left')
test['차량미보유세대수'] = test['차량미보유세대수'].fillna(0)
test.drop(['공급유형', '자격유형'], axis=1, inplace=True)

---

In [34]:
# Train Data의 '임대료' 변수 재생성 (전용면적별세대수 가중 평균)
train['임대료'] = round(train['임대료'] * train['전용면적별세대수'] / train['전용면적별세대수합'])
rental_fee = train.groupby('단지코드', as_index=False).agg({'임대료':'sum'})
train = pd.merge(train, rental_fee, how='left', on='단지코드').drop(['임대료_x'], axis=1).rename(columns={'임대료_y':'임대료'})

In [35]:
# Test Data의 '임대료' 변수 재생성 (전용면적별세대수 가중 평균)
test['임대료'] = round(test['임대료'] * test['전용면적별세대수'] / test['전용면적별세대수합'])
rental_fee = test.groupby('단지코드', as_index=False).agg({'임대료':'sum'})
test = pd.merge(test, rental_fee, how='left', on='단지코드').drop(['임대료_x'], axis=1).rename(columns={'임대료_y':'임대료'})

In [36]:
# Train Data의 '임대보증금' 변수 재생성 (전용면적별세대수 가중 평균)
train['임대보증금'] = round(train['임대보증금'] * train['전용면적별세대수'] / train['전용면적별세대수합'])
rental_fee = train.groupby('단지코드', as_index=False).agg({'임대보증금':'sum'})
train = pd.merge(train, rental_fee, how='left', on='단지코드').drop(['임대보증금_x'], axis=1).rename(columns={'임대보증금_y':'임대보증금'})

In [37]:
# Test Data의 '임대보증금' 변수 재생성 (전용면적별세대수 가중 평균)
test['임대보증금'] = round(test['임대보증금'] * test['전용면적별세대수'] / test['전용면적별세대수합'])
rental_fee = test.groupby('단지코드', as_index=False).agg({'임대보증금':'sum'})
test = pd.merge(test, rental_fee, how='left', on='단지코드').drop(['임대보증금_x'], axis=1).rename(columns={'임대보증금_y':'임대보증금'})

---
#### 소형세대 변수추가

In [38]:
# 소형세대, 중형세대, 대형세대 변수 생성
size= pd.DataFrame(columns=['단지코드','소형세대', '중형세대','대형세대'])
size['단지코드'] = train.단지코드.unique()
code=size['단지코드']

for i in range(len(code)):
    house = train[train['단지코드']==code[i]]
    size.loc[i, '소형세대'] = sum(house.loc[house['전용면적']<40,'전용면적별세대수'])
    size.loc[i, '중형세대'] = sum(house.loc[(house['전용면적']>=40)&(house['전용면적']<80) ,'전용면적별세대수'])
    size.loc[i, '대형세대'] = sum(house.loc[house['전용면적']>=80,'전용면적별세대수'])

size['소형세대'] = size['소형세대'].astype('int64')
size['중형세대'] = size['중형세대'].astype('int64')
size['대형세대'] = size['대형세대'].astype('int64')

train = pd.merge(train, size, on='단지코드', how='left')
train['소형세대'] = train['소형세대']/train['총세대수']
train['중형세대'] = train['중형세대']/train['총세대수']
train['대형세대'] = train['대형세대']/train['총세대수']

# 소형세대, 중형세대, 대형세대 변수 생성
size= pd.DataFrame(columns=['단지코드','소형세대', '중형세대','대형세대'])
size['단지코드'] = test.단지코드.unique()
code=size['단지코드']

for i in range(len(code)):
    house = test[test['단지코드']==code[i]]
    size.loc[i, '소형세대'] = sum(house.loc[house['전용면적']<40,'전용면적별세대수'])
    size.loc[i, '중형세대'] = sum(house.loc[(house['전용면적']>=40)&(house['전용면적']<80) ,'전용면적별세대수'])
    size.loc[i, '대형세대'] = sum(house.loc[house['전용면적']>=80,'전용면적별세대수'])

size['소형세대'] = size['소형세대'].astype('int64')
size['중형세대'] = size['중형세대'].astype('int64')
size['대형세대'] = size['대형세대'].astype('int64')

test = pd.merge(test, size, on='단지코드', how='left')
test['소형세대'] = test['소형세대']/test['총세대수']
test['중형세대'] = test['중형세대']/test['총세대수']
test['대형세대'] = test['대형세대']/test['총세대수']

---

In [39]:
# Train Data의 '전용면적' 변수 재생성 (전용면적별세대수 가중 평균)
train['전용면적'] = round(train['전용면적'] * train['전용면적별세대수'] / train['전용면적별세대수합'])
rental_fee = train.groupby('단지코드', as_index=False).agg({'전용면적':'sum'})
train = pd.merge(train, rental_fee, how='left', on='단지코드').drop(['전용면적_x'], axis=1).rename(columns={'전용면적_y':'전용면적'})

In [40]:
# Test Data의 '전용면적' 변수 재생성 (전용면적별세대수 가중 평균)
test['전용면적'] = round(test['전용면적'] * test['전용면적별세대수'] / test['전용면적별세대수합'])
rental_fee = test.groupby('단지코드', as_index=False).agg({'전용면적':'sum'})
test = pd.merge(test, rental_fee, how='left', on='단지코드').drop(['전용면적_x'], axis=1).rename(columns={'전용면적_y':'전용면적'})

---

In [41]:
# Train Data의 '아파트수' 변수 생성
num_of_apt = pd.DataFrame(train['단지코드'].value_counts()).reset_index().rename(columns={'index':'단지코드', '단지코드':'아파트수'})
train = pd.merge(train, num_of_apt, how='left', on='단지코드')
print(train.shape)

(2307, 27)


In [42]:
# Test Data의 '아파트수' 변수 생성
num_of_apt = pd.DataFrame(test['단지코드'].value_counts()).reset_index().rename(columns={'index':'단지코드', '단지코드':'아파트수'})
test = pd.merge(test, num_of_apt, how='left', on='단지코드')
print(test.shape)

(831, 26)


---

In [43]:
# Train Data의 '차량보유인구비율' 변수 생성
age_gender_info = pd.read_csv('age_gender_info.csv')
cars = pd.DataFrame(age_gender_info.iloc[:, [7, 8, 9, 10, 12, 14]].sum(axis=1), columns=['차량보유인구비율'])
cars['지역'] = age_gender_info['지역']
train = pd.merge(train, cars, how='left', on='지역')
print(train.shape)

(2307, 28)


In [44]:
# Test Data의 '차량보유인구비율' 변수 생성
test = pd.merge(test, cars, how='left', on='지역')
print(test.shape)

(831, 27)


---

In [45]:
train = train.drop(['전용면적별세대수'], axis=1).drop_duplicates().reset_index(drop=True)
train['차량미보유세대비율'] = train['차량미보유세대수'] / train['총세대수']

In [46]:
test = test.drop(['전용면적별세대수'], axis=1).drop_duplicates().reset_index(drop=True)
test['차량미보유세대비율'] = test['차량미보유세대수'] / test['총세대수']

In [47]:
# train = train.drop('총입주민수', axis=1).drop_duplicates().reset_index(drop=True)
# test = test.drop('총입주민수', axis=1).drop_duplicates().reset_index(drop=True)

In [48]:
train.columns

Index(['단지코드', '총세대수', '지역', '공가수', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수', '상가수', '자격유형_major',
       '공급유형_major', '전용면적별세대수합', '전용면적별세대수합_myhome', '입주후기간(월)', '단지명', '주소',
       '총입주민수', '준공후기간(월)', '차량미보유세대수', '임대료', '임대보증금', '소형세대', '중형세대', '대형세대',
       '전용면적', '아파트수', '차량보유인구비율', '차량미보유세대비율'],
      dtype='object')

In [49]:
# '입주후기간(월)' Imputation
from impyute.imputation.cs import mice
train['준공후기간(월)']=train['준공후기간(월)'].astype('float64')
train_num = train.loc[:,train.dtypes!=object]
train_imp = mice(train_num.values)
train_imp = pd.DataFrame(train_imp)
train_imp.columns = train_num.columns
train['준공후기간(월)'] = train_imp['준공후기간(월)']

In [50]:
# '입주후기간(월)' Imputation
from impyute.imputation.cs import mice
test['준공후기간(월)']=test['준공후기간(월)'].astype('float64')
test_num = test.loc[:,test.dtypes!=object]
test_imp = mice(test_num.values)
test_imp = pd.DataFrame(test_imp)
test_imp.columns = test_num.columns
test['준공후기간(월)'] = test_imp['준공후기간(월)']

In [51]:
train.shape

(411, 28)

In [52]:
test.shape

(147, 27)

In [53]:
train.columns

Index(['단지코드', '총세대수', '지역', '공가수', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수', '상가수', '자격유형_major',
       '공급유형_major', '전용면적별세대수합', '전용면적별세대수합_myhome', '입주후기간(월)', '단지명', '주소',
       '총입주민수', '준공후기간(월)', '차량미보유세대수', '임대료', '임대보증금', '소형세대', '중형세대', '대형세대',
       '전용면적', '아파트수', '차량보유인구비율', '차량미보유세대비율'],
      dtype='object')

In [54]:
train.to_csv('new_train3.csv', index=False)
test.to_csv('new_test3.csv', index=False)

In [55]:
train.isnull().sum()
test.isnull().sum()

단지코드                             0
총세대수                             0
지역                               0
공가수                              0
도보 10분거리 내 지하철역 수(환승노선 수 반영)     0
도보 10분거리 내 버스정류장 수               0
단지내주차면수                          0
상가수                              0
자격유형_major                       0
공급유형_major                       0
전용면적별세대수합                        0
전용면적별세대수합_myhome                 0
입주후기간(월)                        47
단지명                              0
주소                               0
총입주민수                            0
준공후기간(월)                         0
차량미보유세대수                         0
임대료                              0
임대보증금                            0
소형세대                             0
중형세대                             0
대형세대                             0
전용면적                             0
아파트수                             0
차량보유인구비율                         0
차량미보유세대비율                        0
dtype: int64

---